In [1]:
from astropy.coordinates import SkyCoord
import os
import pyvo
import warnings
import glob, gzip, shutil
import urllib
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.*")

In [2]:
def WISE(pos,size = 0.1):
    """
    This function simplifies the process of WISE image retrieving.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    size : float, optional
        The searching cone size, by default 0.1 [deg]
        Note that the size is NOT the image size.
    """

    # Initiate pyvo service and select allwise as our catalogue
    image_service = pyvo.regsearch(servicetype='image', keywords=['allwise'])
    image_table = image_service[0].search(pos=pos, size=size) # index 0 is fixed according to regsearch result
    im_table = image_table.to_table()

    # Preventing from repeating download
    a = im_table.to_pandas()
    b = a[a['sia_fmt'].str.contains('fits')].drop_duplicates(subset=['sia_url']) # filter for fits file, exclude html files

    # Start downloading
    for _, row in b.sort_values(['sia_bp_id','coadd_id']).iterrows():
        no = 0

        while os.path.exists(f"{row.sia_bp_id}_{no}.fits.gz"): # file numbering
            no += 1
        urllib.request.urlretrieve(f"{row.sia_url}?center={pos.ra.degree},{pos.dec.degree}&size=800pix", f"{row.sia_bp_id}_{no}.fits.gz") 
        urllib.request.urlretrieve(f"{row.unc_url}?center={pos.ra.degree},{pos.dec.degree}&size=800pix", f"{row.sia_bp_id}_unc_{no}.fits.gz") 
        # Download images from IRSA, Here we do crop by size in the url, otherwise the WISE image would be too large

    # unzip files
    for file in glob.glob("*.gz"):
        with gzip.open(file, 'r') as f_in, open(file.replace(".gz", ""), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)  # unzipped gz files
        os.remove(file)   # delete gz files   

In [3]:
WISE(SkyCoord.from_name("SN 2002cd"))